In [25]:
import pandas as pd
import json

In [26]:
with open('mysql.json', 'r') as file:
    config_str = file.read()
config = json.loads(config_str)

In [27]:
import pymysql
conn = pymysql.connect(
    host = config['host'],
    user = config['user'],
    password = config['password'],
    database = config['database'],
    port = config['port']
)

In [4]:
conn = pymysql.connect(**config)

In [28]:
import random
random.seed(2021)

## products table 생성

In [29]:
cur = conn.cursor()
sql_table = '''
    CREATE TABLE if NOT EXISTS products(
    pid INT PRIMARY KEY AUTO_INCREMENT,
    pname VARCHAR(40) NOT NULL,
    pprice INT UNSIGNED NOT NULL,
    pcategory VARCHAR(20) NOT NULL,
    pcost INT UNSIGNED NOT NULL
) AUTO_INCREMENT=11;
'''

In [30]:
cur.execute(sql_table)

0

In [31]:
# 제품 리스트
electric_products = ['삼성노트북','LG노트북','레노버노트북','Apple노트북','삼성스마트폰','아이폰','헤드셋','스피커','마우스','키보드']
car_products = ['블랙박스','내비게이션','하이패스','후방카메라','자동차TV','카오디오','핸즈프리','헤드유닛','방진매트','윤활방청제']
sport_products = ['스포츠의류','운동화','수영','구기','라켓','골프','자전거','보드','캠핑','낚시']

In [32]:
sql = '''INSERT INTO products(pname, pprice, pcategory,pcost) VALUES(%s,%s,%s,%s);'''
category = ['전자제품','자동차용품','스포츠용품']
for i, items in enumerate([electric_products,car_products,sport_products]):
    for item in items:
        price = random.randint(10,999)*100
        rate = random.randint(89,95)
        cost = int(round(price*rate/1000)*10)
        cur.execute(sql,(item,price,category[i],cost))

conn.commit()

In [33]:
sql = '''
    CREATE TABLE if NOT EXISTS sales(
    sid INT PRIMARY KEY auto_increment,
    sdate DATE NOT NULL,
    scompany VARCHAR(20) NOT NULL,
    spid INT NOT NULL,
    sunit INT NOT NULL,
    FOREIGN KEY(spid) REFERENCES products(pid)
) AUTO_INCREMENT=101;
'''

In [34]:
cur.execute(sql)

0

In [35]:
sql_sale_insert = '''
    INSERT INTO sales VALUES(DEFAULT,%s,%s,%s,%s);
'''

In [36]:
company_list = ['삼성','LG','한화','포스코','GS']
for i in range(1,13):
    if i in [1,3,5,7,8,10,12]:
        for k in range(10):
            date = f'2021-{i}-{random.randint(1,31)}'
            company = random.choice(company_list)
            unit = random.randint(1,20)
            pid = random.randint(11,40)
            cur.execute(sql_sale_insert,(date,company,pid,unit))
    elif i ==2:
        for k in range(10):
            date = f'2021-{i}-{random.randint(1,29)}'
            company = random.choice(company_list)
            unit = random.randint(1,20)
            pid = random.randint(11,40)
            cur.execute(sql_sale_insert,(date,company,pid,unit))
    else:
        for k in range(10):
            date = f'2021-{i}-{random.randint(1,30)}'
            company = random.choice(company_list)
            unit = random.randint(1,20)
            pid = random.randint(11,40)
            cur.execute(sql_sale_insert,(date,company,pid,unit))

In [37]:
conn.commit()

In [38]:
sql = '''
    CREATE VIEW sales_book as
    SELECT sid, sdate, scompany, pid, pname, pcategory, pprice, sunit, pcost,
    pprice*sunit AS revenue, (pprice-pcost)*sunit AS profit
    FROM sales
    JOIN products
    ON sales.spid = products.pid
    ORDER BY sdate;
'''

In [39]:
cur = conn.cursor()
cur.execute(sql)

0

In [40]:
sql = 'select * from sales_book'
cur.execute(sql)
results = cur.fetchall()

In [41]:
df = pd.DataFrame(results)
df.head()

,0,1,2,3,4,5,6,7,8,9,10
0,103,2021-01-01,GS,17,헤드셋,전자제품,7500,8,6820,60000,5440
1,107,2021-01-03,삼성,39,캠핑,스포츠용품,49100,6,43700,294600,32400
2,109,2021-01-05,포스코,35,라켓,스포츠용품,82900,13,76270,1077700,86190
3,101,2021-01-08,포스코,20,키보드,전자제품,94500,16,85050,1512000,151200
4,110,2021-01-09,포스코,31,스포츠의류,스포츠용품,99000,6,94050,594000,29700


In [42]:
del df[3]
del df[0]

In [43]:
df.columns = ['판매날짜','고객사','상품명','카테고리','단가','수량','원가','매출액','이익']
df.head()

,판매날짜,고객사,상품명,카테고리,단가,수량,원가,매출액,이익
0,2021-01-01,GS,헤드셋,전자제품,7500,8,6820,60000,5440
1,2021-01-03,삼성,캠핑,스포츠용품,49100,6,43700,294600,32400
2,2021-01-05,포스코,라켓,스포츠용품,82900,13,76270,1077700,86190
3,2021-01-08,포스코,키보드,전자제품,94500,16,85050,1512000,151200
4,2021-01-09,포스코,스포츠의류,스포츠용품,99000,6,94050,594000,29700


In [44]:
df.to_csv('sales_book.csv', index=False)

In [45]:
cur.close()
conn.close()